<a href="https://colab.research.google.com/github/kozyreviva/Climat_DS/blob/main/Air_Indicators/climat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
!pip install dataprep



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [118]:
# import libraris
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dataprep.clean import clean_country

In [119]:
def new_col_name(data):
   colnames = list(data.iloc[0])
   colnames[0] = 'country_id'
   colnames[1] = 'country'
   colnames[-3:] = data.columns[-3:]
   data.columns = colnames
   return  data.drop(0).reset_index(drop = True)


In [120]:
def replace_to_na(data):
    return data.replace(['...','…'], np.NaN)

In [121]:
def change_type(data):
    data[data.columns[2:30]] = data[data.columns[2:30]].replace(to_replace = ',', value = '', regex = True)
    data[data.columns[2:30]] = data[data.columns[2:30]].astype(float)
    data[data.columns[31]] = data[data.columns[31]].astype('Int64')
    data[data.columns[32]] = data[data.columns[32]].replace(to_replace = ',', value = '', regex = True)
    data[data.columns[32]] = data[data.columns[32]].astype(float)
    data[data.columns[33]] = data[data.columns[33]].replace(to_replace = ',', value = '', regex = True)
    data[data.columns[33]] = data[data.columns[33]].astype(float)

In [122]:
def country_change_alpha(data):
    new_data = (data[['country','% change since 1990']]
                .sort_values('% change since 1990',ascending = False)
                .reset_index(drop = True)
                .head(10)
                .copy())
    new_data = clean_country(new_data, "country", output_format = 'alpha-3')
    new_data = new_data.rename(columns = {'country_clean':'iso_alpha'})
    return new_data



In [123]:
def change_data_by_years(data):
    lst =[]
    for i, row in df_n2o.iloc[:,2:31].items():
        for item in row:
            lst.append([i,item])
    df = pd.DataFrame(lst)
    df = df.rename(columns = {0:'year',1:'values'})

    new_n2o_country = data['country'].copy()
    for i in range(28):
       new_n2o_country = pd.concat([new_n2o_country,data['country']], ignore_index = True)

    new_df = df.join(new_n2o_country)
    new_df = clean_country(new_df, "country", output_format = 'alpha-3')
    new_df = new_df.rename(columns = {'country_clean':'iso_alpha'})
    return new_df

In [124]:
def to_bar(data):
  fig = px.bar(data, x = "country", y = "% change since 1990")
  fig.show()

In [125]:
def percents_map(data):
    fig = px.scatter_geo(data,
                         locations = "iso_alpha",
                         size = "% change since 1990",
                         projection="natural earth",
                         color= "country")
    fig.show()

In [146]:
def values_to_map(data):
    fig = px.choropleth(data,
                        locations="iso_alpha",
                        color="values",
                        hover_name="country",
                        animation_frame="year"
                        )
    fig.update_layout(title_text = 'Animated_map',
                      title_x = 0.5,
                      geo=dict(showframe = False,
                               showcoastlines = False
                               )
                      )
    fig.show()

In [127]:
# read datasets
df_n2o = pd.read_csv("N2O_Emissions.csv")
df_co2 = pd.read_csv("CO2_Emissions.csv")
df_so2 = pd.read_csv("SO2_emissions.csv")
df_ch4 = pd.read_csv("CH4_Emissions.csv")
df_ghg = pd.read_csv("GHG_Emissions.csv")
df_nox = pd.read_csv("NOx_Emissions.csv")

In [128]:
df_n2o.head(10)

,Country ID,Country,"Time series - Total N2O emissions, in 1000 tonnes of CO2 equivalent",Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,"N2O emissions, latest year",% change since 1990,"N2O emissions \nper capita, \nlatest year"
0,NaN,NaN,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,Latest Year,1000 tonnes of \nCO2 equivalent,%,kg
1,4.0,Afghanistan,...,...,...,...,...,...,...,...,...,"19,610.00",...,...,...,...,...,2013,"19,610.00",...,0.608
2,8.0,Albania,96.10,136.40,108.50,96.10,96.10,93.00,83.70,77.50,...,...,...,...,...,...,...,2009,120.96,25.87,0.041
3,12.0,Algeria,...,...,...,...,"9,300.00",...,...,...,...,...,...,...,...,...,...,2000,"6,500.70",...,0.209
4,24.0,Angola,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,2005,"13,869.40",...,0.714
5,28.0,Antigua and Barbuda,1.58,...,...,...,...,...,...,...,...,...,...,...,...,...,...,2000,83.70,"5,194.12",1.101
6,32.0,Argentina,"51,823.68",...,...,...,"56,689.61",...,...,"62,089.62",...,...,...,...,...,...,...,2012,"73,154.41",41.16,1.752
7,51.0,Armenia,169.57,...,...,...,...,...,...,...,...,...,...,...,...,...,...,2010,484.07,185.47,0.168
8,36.0,Australia,53.48,52.21,51.57,53.14,54.75,52.30,56.55,59.22,...,65.95,67.44,65.63,65.66,71.36,67.50,2018,67.50,26.20,0.003
9,40.0,Austria,14.50,14.61,13.98,14.08,13.92,14.24,14.34,14.43,...,11.53,11.81,11.84,12.15,11.95,11.83,2018,11.83,-18.39,0.001


In [129]:
df_n2o = new_col_name(df_n2o)
df_co2 = new_col_name(df_co2)
df_ghg = new_col_name(df_ghg)
df_so2 = new_col_name(df_so2)
df_nox = new_col_name(df_nox)
df_ch4 = new_col_name(df_ch4)

In [130]:
df_n2o = replace_to_na(df_n2o)
df_co2 = replace_to_na(df_co2)
df_ghg = replace_to_na(df_ghg)
df_so2 = replace_to_na(df_so2)
df_nox = replace_to_na(df_nox)
df_ch4 = replace_to_na(df_ch4)

In [131]:
change_type(df_n2o)
change_type(df_co2)
change_type(df_ghg)
change_type(df_so2)
change_type(df_nox)
change_type(df_ch4)

In [132]:
df_n2o.head(10)

,country_id,country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,Latest Year,"N2O emissions, latest year",% change since 1990,"N2O emissions \nper capita, \nlatest year"
0,4.0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19610.00,NaN,NaN,NaN,NaN,NaN,2013,19610.00,NaN,0.608
1,8.0,Albania,96.10,136.40,108.50,96.10,96.10,93.00,83.70,77.50,...,NaN,NaN,NaN,NaN,NaN,NaN,2009,120.96,25.87,0.041
2,12.0,Algeria,NaN,NaN,NaN,NaN,9300.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2000,6500.70,NaN,0.209
3,24.0,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2005,13869.40,NaN,0.714
4,28.0,Antigua and Barbuda,1.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2000,83.70,5194.12,1.101
5,32.0,Argentina,51823.68,NaN,NaN,NaN,56689.61,NaN,NaN,62089.62,...,NaN,NaN,NaN,NaN,NaN,NaN,2012,73154.41,41.16,1.752
6,51.0,Armenia,169.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2010,484.07,185.47,0.168
7,36.0,Australia,53.48,52.21,51.57,53.14,54.75,52.30,56.55,59.22,...,65.95,67.44,65.63,65.66,71.36,67.50,2018,67.50,26.20,0.003
8,40.0,Austria,14.50,14.61,13.98,14.08,13.92,14.24,14.34,14.43,...,11.53,11.81,11.84,12.15,11.95,11.83,2018,11.83,-18.39,0.001
9,31.0,Azerbaijan,3772.70,1925.10,2132.80,1701.90,1429.10,1295.80,1308.20,1515.90,...,2767.12,NaN,NaN,NaN,NaN,NaN,2013,2767.12,-26.65,0.295


In [133]:
n2o_country_change = country_change_alpha(df_n2o)
co2_country_change = country_change_alpha(df_co2)
ghg_country_change = country_change_alpha(df_ghg)
so2_country_change = country_change_alpha(df_so2)
nox_country_change = country_change_alpha(df_nox)
ch4_country_change = country_change_alpha(df_ch4)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.



Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.



Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.



Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.



Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


Country Cleaning Report:
	10 values cleaned (100.0%)
Result contains 10 (100.0%) values in the correct format and 0 null values (0.0%)


In [134]:
to_bar(n2o_country_change)
to_bar(co2_country_change)
to_bar(ghg_country_change)
to_bar(so2_country_change)
to_bar(nox_country_change)
to_bar(ch4_country_change)

In [135]:
percents_map(n2o_country_change)

In [136]:
n2o_data_by_years = change_data_by_years(df_n2o)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:6751: FutureWarning:

Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.

                                     

Country Cleaning Report:
	5452 values cleaned (100.0%)
Result contains 5452 (100.0%) values in the correct format and 0 null values (0.0%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5452 entries, 0 to 5451
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   year       5452 non-null   object
 1   values     2066 non-null   object
 2   country    5452 non-null   object
 3   iso_alpha  5452 non-null   object
dtypes: object(4)
memory usage: 170.5+ KB


In [145]:
values_to_map(n2o_data_by_years)